<br>Developed by: Thomas.Reimann@tu-dresden.de
<br>Last change: 2021 02 13
<br>Current state: Funktional / ggf. noch optimieren

<center>
    
### MWW01 - Grundwasserbewirtschaftung mit Computermodellen

### Thema 04: Numerische Lösung des Stofftransports

# Ausweisung von Schutzzonen / Abschätzung des Einzugsgbietes eines Brunnens mit der Trennstromlinie  </center>

### Lernziele:
Nachdem Sie dieses Notebook erfolgreich bearbeitet haben, können Sie:
* die Trennstromlinie mit Hilfe einer analytischen Gleichung berechnen,
* erläutern, wie sich die Parameter (bzw. die Unsicherheit, mit der die Parameter bestimmt werden) auf die Trennstromlinie auswirken,
* die Wasserentnahme im Falle einer Havarie / Kontamination so optimieren, dass die Qualität des gefassten Rohwassers möglichst nicht beeinträchtigt wird.

### Ausgangssituation
Betrachtet wird eine Wasserfassung, welche sich in einem gespannten, homogenen und isotropen Grundwasserleiter befindet. Bei Wasserentnahme bildet sich die Trennstromlinie der Wasserfassung. Die Trennstromlinie wird durch zwei charakteristische Größen gekennzeichnet (Abbildung 1):
* der Kulminationspunkt x_0 sowie
* die Breite B der Trennstromlinie.

<br>
<center>
$\large x_0 = -\frac{Q}{2\pi Kib}$</center>

<br>
<center>
$\large B = 2y_{max} = \frac{Q}{Kib}$</center>

<br>
<center>
$\large y_{max} = \pm \frac{Q}{2KiB}$</center>

<br>
<figure>
  <img src="FIG/MWW01_T04_TSL_1.jpg" alt="Trennstromlinie mit charakteristischen Größen" style="width:50%">
  <figcaption>Abb.1 - Trennstromlinie einer Wasserfassung mit charakteristischen Größen.</figcaption>
</figure>

Weiterhin kann die Form der Trennstromlinie mit Hilfe der analytischen Gleichung nach <a href="https://ngwa.onlinelibrary.wiley.com/doi/10.1111/j.1745-6584.1993.tb00824.x" target="_blank">Grubb (1993)</a> beschrieben werden.

<br>
<center>
$\large x = \frac{-y}{tan(\frac{2\pi K i b y}{Q})}$</center>

<br>
Nachfolgend werden die nötigen Python-Bibliotheken / Funktionen geladen, um die Rechnungen durchzuführen und zu visualisieren.

In [ ]:
# Initialize librarys
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import math
from ipywidgets import *
from math import pi, tan

In [34]:
# Function for catchment width (maximale Breite des Einzugsgebietes)
def ymax_conf(Q, K, i, b):
    ymax = Q/(2.*K*i*b)
    return ymax

# Function for the culmination point (Kulminationspunkt)
def x0_conf(Q, K, i, b):
    x0 = -Q/(2.*np.pi*K*i*b)
    return x0

# Computaton of the well catchment (Berechnung der Trennstromlinie)
def TSL_conf(x_max, Q, K, i, b, x_scale, y_scale, x_Punkt, y_Punkt):
    ymax = ymax_conf(Q, K, i, b)
    x0   = x0_conf(Q, K, i, b)
    x = np.arange(int(x0)-1, x_max,1)
    y = np.arange(-ymax, ymax, 0.1)
    
    # Compute catchment
    x = -1*y/(np.tan(2*np.pi*K*i*b*y/Q))
    
    x_plot = 500 * x_scale
    y_plot = 1000 * y_scale
    
    # Plot
    fig = plt.figure(figsize=(12,8))
    ax = fig.add_subplot(1, 1, 1)

    ax.plot(x,y, label='Trennstromlinie')
    plt.plot(x_Punkt,y_Punkt, marker='o', color='r',linestyle ='None', label='Schadstoffquelle') 
    ax.set(xlabel='x', ylabel='y',title='Brunnen-EZG')
    ax.set(xlim=(-x_plot, 10*x_plot), ylim=(-y_plot, y_plot))
    plt.fill_between(x,y,color='blue', alpha=.1)
    ax.grid()
    plt.legend()
    plt.show()
    
    print('y_max', ymax)
    print('x0', x0)

### Aufgabe 1

Bestimmen Sie die Einzugsgebietsform eines Entnahmebrunnens. Benutzen Sie die nachfolgende Funktionalität.
*	gespannter Aquifer
*	Hydraulische Leitfähigkeit	K = 457,5 m/d
*	Hydraulischer Gradient		I = 0,003 [-]
*	Mächtigkeit			        b = 23 m
*	Entnahmerate			    Q = 4320 m3/d

Ermitteln Sie:  
**a)**	Maximale Breite *B* des Einzugsgebiets  
**b)**	Abstand *x_0* zum Kulminationspunkt  
**c)**	Form der Kurve, die das Einzugsgebiet beschreibt, d. h. die Trennstromlinie

Die nachfolgende Box realisiert die Berechung und visualisiert das Ergebnis. Der Wertebereich des Diagramms kann mit x_scale und y_scale angepasst werden.

In [35]:
x_max = 1000      # Interne Variable um den Zahlenbereich zu definieren
Q = 4320 / 86400  # Entnahmerate, umgerechnet in m und sek
K = 457.5 / 86400 # hydraulische Leitfähigkeit in m und sek         
i = 0.003         # Gradient der (Grund)Strömung
b = 23            # Mächtigkeit des homogenen und isotropen Grundwasserleiters

interact(TSL_conf,
         x_max=fixed(x_max),
         Q=widgets.FloatText        (value=Q, min = 0.001, max = 1, step = 0.01),
         K=widgets.FloatLogSlider   (value=K,base=10,min=-6, max=-2, step=0.1,readout=True,readout_format='.2e'),
         i=widgets.FloatText        (value=i,min = 0.0001, max=0.1, step=0.001,readout=True,readout_format='e'),
         b=widgets.FloatText        (value=b,min = 1,max = 200, step = 1),
         x_scale = widgets.FloatText(value=0.1,min = 0.1,max = 10, step = 0.1),
         y_scale = widgets.FloatText(value=0.1,min = 0.1,max = 10, step = 0.1),
         x_Punkt= widgets.FloatText (value=0, step=10),
         y_Punkt=widgets.FloatText  (value=0,step=10))

interactive(children=(FloatText(value=0.05, description='Q', step=0.01), FloatLogSlider(value=0.00529513888888…

<function __main__.TSL_conf(x_max, Q, K, i, b, x_scale, y_scale, x_Punkt, y_Punkt)>

### Aufgabe 2
Sie können nun die einzelnen Parametern der Trennstromlinie variieren um zu untersuchen, wie sich diese Änderungen auf die Form der Trennstromlinie (auch Breite der TSL und Abstand Fassung zum Kulminationspunkt).  
Passen Sie ggf. die Skalierung der x- und y-Achse des Diagramms an (x_scale / y_scale).  
Beantworten Sie u. a. die Fragen, welche Auswirkungen
* ein kleinerer Gradient,
* ein größerer Gradient etc. 
auf die Form der Trennstromlinie haben. Damit können Sie auch einschätzen, welche Auswirkungen Unsicherheiten bei den Eingangsparametern auf die Form und Lage der Trennstromlinie haben.


### Aufgabe 3 a

Ein Wasserversorger entnimmt aus einem gespannten Aquifer Grundwasser. Folgende Angaben stehen
Ihnen zur Verfügung:
* Entnahmerate (Q) = 2800 m3d-1
* hydraulische Leitfähigkeit K = 2,8 × 10-4 ms-1
* wassererfüllte Mächtigkeit b = 14 m
* natürlicher Gradient des Grundwassers (i) = 0,0012

Berechnen Sie mit Hilfe der nächsten Box die Trennstromlinie (Hinweis: Rechnen Sie die Parameter um und geben Sie diese in den nachfolgenden Felder ein. Sie können den jeweils initialen Wert per Maus / Tastatur verändern. Modifizieren Sie am Ende die Skalierung des Diagramms so, dass Sie die TSL komplett visualisieren können.)

In [36]:
interact(TSL_conf,
         x_max=fixed(x_max),
         Q=widgets.FloatText        (value=0.05, min = 0.001, max = 1, step = 0.01),
         K=widgets.FloatLogSlider(value=1E-3,base=10,min=-6, max=-2, step=0.1,readout=True,readout_format='.2e'),
         i=widgets.FloatText(value=0.001,base=10,min=-4, max=-0, step=0.001,readout=True,readout_format='e'),
         b= widgets.FloatText(value=30,min = 1,max = 200, step = 1),
         x_scale = widgets.FloatText(value=1,min = 0.1,max = 10, step = 0.1),
         y_scale = widgets.FloatText(value=1,min = 0.1,max = 10, step = 0.1),
         x_Punkt= widgets.FloatText(value=0, step=10),
         y_Punkt=widgets.FloatText(value=0,step=10))

interactive(children=(FloatText(value=0.05, description='Q', step=0.01), FloatLogSlider(value=0.001, descripti…

<function __main__.TSL_conf(x_max, Q, K, i, b, x_scale, y_scale, x_Punkt, y_Punkt)>

### Aufgabe 3 b

Im Einzugsgebiet der Wasserfassung kam es in einem Randbereich am Punkt x = 1255 m / y = 860 m durch eine Havarie zum Eintrag
wassergefährdender Stoffe in den Untergrund. Schlagen Sie eine konkrete Sofortmaßnahme vor, um eine unmittelbare Gefahr für die Wasserfassung abzuwenden. Geben Sie dazu die Koordinaten des Schadens in die Visualisierung der TSL ein (x_Punkt / y_Punkt) und optimieren Sie anschliessend die TSL so, dass die unmittelbare Gefahr für die Wasserfassung abgewendet wird.